In [10]:
import pandas as pd

# Load datasets
final_reduced = pd.read_csv('final_reduced_dataset.csv')
all_data = pd.read_csv('all_data.csv')

# Function to find matching column
def find_column(df, possible_names):
    for name in possible_names:
        if name in df.columns:
            return name
    return None

# Possible column name variations (add more if needed)
solute_possibilities = [
    'solute_inchikey', 'inchikey', 'InChIkey', 'solute_InChIKey', 
    'Solute_InChIKey', 'inchi_key', 'InChI_Key', 'mol_id'
]
solvent_possibilities = [
    'Solvent', 'solvent', 'Solvent_Name', 'solvent_name', 
    'solvent_id', 'solvent_name'
]

# Find actual column names
solute_final = find_column(final_reduced, solute_possibilities)
solvent_final = find_column(final_reduced, solvent_possibilities)
solute_all = find_column(all_data, solute_possibilities)
solvent_all = find_column(all_data, solvent_possibilities)

# Verify we found all necessary columns
if not all([solute_final, solvent_final, solute_all, solvent_all]):
    print("ERROR: Could not find all required columns.")
    print(f"final_reduced columns: Solute - {solute_final}, Solvent - {solvent_final}")
    print(f"all_data columns: Solute - {solute_all}, Solvent - {solvent_all}")
    print("\nActual columns in final_reduced:", final_reduced.columns.tolist())
    print("Actual columns in all_data:", all_data.columns.tolist())
else:
    print(f"Using columns:\n"
          f"final_reduced: {solute_final} (solute), {solvent_final} (solvent)\n"
          f"all_data: {solute_all} (solute), {solvent_all} (solvent)")

    # Create mapping dictionary (case insensitive)
    final_reduced['key1'] = final_reduced[solute_final].astype(str).str.lower().str.strip()
    final_reduced['key2'] = final_reduced[solvent_final].astype(str).str.lower().str.strip()
    logS_map = final_reduced.set_index(['key1', 'key2'])['LogS'].to_dict()

    # Add LogS column with case insensitive matching
    all_data['temp_key1'] = all_data[solute_all].astype(str).str.lower().str.strip()
    all_data['temp_key2'] = all_data[solvent_all].astype(str).str.lower().str.strip()
    
    # Track missing matches
    missing = []
    
    def get_logs(row):
        key = (row['temp_key1'], row['temp_key2'])
        if key not in logS_map:
            missing.append(key)
            return None
        return logS_map[key]
    
    all_data['LogS'] = all_data.apply(get_logs, axis=1)
    
    # Clean up temporary columns
    all_data.drop(['temp_key1', 'temp_key2'], axis=1, inplace=True)
    
    # Save results
    all_data.to_csv('all_data_with_LogS.csv', index=False)
    
    # Report statistics
    total = len(all_data)
    matched = total - len(missing)
    print(f"\nSuccessfully matched {matched}/{total} rows ({matched/total:.1%})")
    
    if missing:
        print(f"{len(missing)} rows had no matching LogS value")
        # Save missing keys for investigation
        pd.DataFrame(missing, columns=['solute_key', 'solvent_key'])\
          .to_csv('missing_logs_matches.csv', index=False)
        print("Missing matches saved to missing_logs_matches.csv")

Using columns:
final_reduced: solute_inchikey (solute), Solvent (solvent)
all_data: InChIkey (solute), Solvent (solvent)

Successfully matched 3249/3249 rows (100.0%)


In [11]:
import pandas as pd

# Load the merged data
merged_data = pd.read_csv('all_data_with_LogS.csv')

# Check for null values in LogS column
null_logs = merged_data['LogS'].isnull().sum()
print(f"Number of rows with missing LogS values: {null_logs}")

# Show sample rows with missing LogS
if null_logs > 0:
    print("\nSample rows with missing LogS:")
    print(merged_data[merged_data['LogS'].isnull()].head())

Number of rows with missing LogS values: 43

Sample rows with missing LogS:
                        InChIkey       Solvent       MW     SASA      G_sol  \
50   CIWBSHSKHKDKBQ-JLAZNSOCNA-N       toluene  176.126  180.498 -24465.056   
93   ZYVHVGJGLOEEKD-FQFUPTBWNA-N       toluene  286.243  296.542 -30777.268   
224  BXAOUWIVXLQYOZ-SPEPDGBUNA-N   cyclohexane  443.542  446.815 -44890.280   
233  GSBSDUZVSCTUKA-PKSOQXRJNA-N        hexane  288.345  298.929 -28673.942   
287  GSBSDUZVSCTUKA-PKSOQXRJNA-N  acetonitrile  288.345  305.724 -29034.254   

      DeltaG_sol   Volume  sol_dip  LsoluHsolv  LsolvHsolu  O_charges  \
50   -534.012500  236.332  2.65075       8.161      10.370   -2.78640   
93  -1081.589000  311.673  4.43134       7.288      10.665   -2.63331   
224  -304.913867  239.587  1.67927      10.171      13.314   -2.45614   
233     2.388933  182.922  4.16798       9.706      12.450   -1.05470   
287  -357.923067  346.342  3.45279      11.302       9.546   -1.11621   

     C_cha

In [14]:
# Calculate average LogS by solvent
solvent_avg = merged_data.groupby('Solvent')['LogS'].mean()

# Fill missing LogS values with solvent averages
merged_data['LogS'] = merged_data.apply(
    lambda row: solvent_avg[row['Solvent']] if pd.isnull(row['LogS']) else row['LogS'],
    axis=1
)

# Save the final data
merged_data.to_csv('all_data_with_LogS_final.csv', index=False)

In [15]:
import pandas as pd

# Load the merged data
merged_data = pd.read_csv('all_data_with_LogS_final.csv')

# Check for null values in LogS column
null_logs = merged_data['LogS'].isnull().sum()
print(f"Number of rows with missing LogS values: {null_logs}")

# Show sample rows with missing LogS
if null_logs > 0:
    print("\nSample rows with missing LogS:")

Number of rows with missing LogS values: 0
